<a href="https://colab.research.google.com/github/viriditass/TIL/blob/main/Very_Deep_Convolutional_Networks_for_Large_Scale_Image_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Very Deep Convolutional Networks for Large-Scale Image Recognition**

논문 링크: https://arxiv.org/abs/1409.1556